# Introduction:
5G, the fifth generation of radio technology, has brought about new services, technologies, and networking paradigms, with the corresponding social benefits. However, there is growing concern over the energy consumption of these new network deployments. While 5G networks are estimated to be about 4x more energy-efficient than 4G networks, their energy consumption is approximately 3x larger due to the need for a larger number of cells to provide the same coverage at higher frequencies and the increased processing required for wider bandwidths and more antennas. It is worth noting that, on average, network operational expenditure (OPEX) already accounts for around 25 percent of the total operator’s cost, and 90 percent of it is spent on large energy bills. More than 70 percent of this energy is estimated to be consumed by the radio access network (RAN), particularly by the base stations (BSs), while data centres and fibre transport account for a smaller share.(https://zindi.africa/competitions/aiml-for-5g-energy-consumption-modelling)

## Objectives:

**Objective A: Estimating Energy Consumption in Specific Base Station Products:** For this objective, it's understood that specific base information, their configurations, and a historical usage, typically spanning a complete week, were received. In this scenario, the training data, and primarily the history of energy expenditure, were used to determine the best forecasting patterns. It was found that treating this as a time-series problem yielded the best results. The model uses energy and load along with the seasonality of each base's use to pinpoint the most accurate energy estimates.

**Objective B: Generalization Across Different Base Station Products:** In this scenario, I understood that having a history of bases with similar configurations enables using that history to forecast a new base that uses similar configurations. Here, we face a different problem because the user usage history and its hourly seasonality become irrelevant, as does the energy history. It's still possible to treat it as a time-series problem by using lags and other such features. However, what showed the best result was the use of complex features concerning the new bases' configurations. This shows that there still exists a way to use historical data to forecast unknown futures and that they still have a certain strong correlation. The issue transforms into a hybrid problem, combining elements of time-series analysis and complex methods to identify patterns in new base configurations for precise energy prediction.

**ObjC - Generalization across different base station configurations:** In this scenario, it's understood that besides new databases, the model should also generalize to new base configurations, often with significant differences, making it practically impossible to use other bases' history for forecasting energy consumption. After several optimizations and tests, it was understood that there wasn't a possibility to create a complex model and that the history of data did not hold much relevance. Characteristics such as load, EMS Modes 1, 2, and 6 gained importance. It was noticed that the simpler the model, the better it performed and the more general it became. Therefore, all robust features and all time-series features were removed, generating greater generalization power and avoiding overfitting.

# Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_absolute_error
import xgboost as xgb
from xgboost import DMatrix, train
from catboost import CatBoostRegressor, Pool
pd.set_option("display.max_columns",None)
np.random.seed(42)
import math
import random

# Data Cleaning

In [ ]:
pcp_df = pd.read_csv("/kaggle/input/reseau5g/imgs_202307101549519358.csv")
cl_df = pd.read_csv("/kaggle/input/reseau5g/imgs_2023071012130978799.csv")
bs_df = pd.read_csv("/kaggle/input/reseau5g/imgs_2023071012123392536.csv")
ec_df  = pd.read_csv("/kaggle/input/reseau5g/imgs_2023071012133740345.csv")
ss_df = pd.read_csv("/kaggle/input/reseau5g/SampleSubmission (24).csv")
pcp_df.shape, cl_df.shape, bs_df.shape, ec_df.shape, ss_df.shape

((26139, 4), (125575, 10), (1217, 8), (92629, 3), (26139, 2))

In [ ]:
cl_bs_df = cl_df.merge(bs_df, on=['BS', 'CellName'], how='left')
cl_bs_df = cl_bs_df.pivot(
    index=['Time','BS'],columns=['CellName'],
    values=['CellName']
).reset_index()
cl_bs_df.columns = ['_'.join([str(i) for i in x]) for x in cl_bs_df.columns]
cl_bs_df.columns = cl_bs_df.columns.str.strip('_')
df= cl_bs_df.merge(bs_df.groupby('BS')[['RUType','Mode','Frequency','Antennas','Bandwidth']].first().reset_index(), on='BS', how='left')
df= df.merge(cl_df.groupby(['BS','Time'])[['load','ESMode1','ESMode2','ESMode3','ESMode4','ESMode5','ESMode6']].sum().reset_index(), on=['BS',"Time"], how='left')
df= df.merge(bs_df.groupby('BS')[['TXpower']].max().reset_index(), on='BS', how='left')
df=df.fillna(0)
df['CellName_Cell1'].replace({'Cell1': 1}, inplace=True)
df['CellName_Cell2'].replace({'Cell2': 1}, inplace=True)
df = df.merge(ec_df, on=['Time', 'BS'], how='left')

In [ ]:
plt.figure(figsize=(20, 5))
sns.lineplot(data=df, x='Time', y='Energy')
plt.show()

In [ ]:
plt.figure(figsize=(8, 4))
sns.barplot(df, x='RUType', y="Energy")
plt.show()

In [ ]:
plt.figure(figsize=(20, 5))
sns.lineplot(data=df, x='Time', y='Energy', hue='Mode')
plt.show()

# Feature Engeniring

In [ ]:
def Feature_Engineering(df):
    df['Time'] = pd.to_datetime(df['Time'])
    df['Hour'] = df['Time'].dt.hour
    df['Day'] = df['Time'].dt.day
    df['Dayofweek'] = df['Time'].dt.dayofweek
    df["periode"]=None
    for i in range(118768):
        if 0 <= df.loc[i, 'Hour'] <= 5:
            df.loc[i, 'periode'] = 0
        elif 6 <= df.loc[i, 'Hour'] < 10:
            df.loc[i, 'periode'] = 1
        elif 11 <= df.loc[i, 'Hour'] < 20:
            df.loc[i, 'periode'] = 2
        else:
            df.loc[i, 'periode'] = 3
    label_encoder = LabelEncoder()
    df['ID'] = df['Time'].astype('str') + '_' + df['BS']
    df['BS'] = df['BS'].apply(lambda x: int(x.strip('B_')))
    df['periode'] = label_encoder.fit_transform(df['periode'])
    df['RUType'] = label_encoder.fit_transform(df['RUType'])
    df['Mode'] = label_encoder.fit_transform(df['Mode'])
    df = pd.get_dummies(df, columns=['RUType','Day','Hour','Mode'],drop_first=True,dtype=float)
    df=df.sort_values(by=['BS', 'Time'], ascending=[True, True]).reset_index(drop=True)
    BS_mean = df.groupby('BS')['load'].mean().to_dict()
    BS_std = df.groupby('BS')['load'].std().to_dict()
    BS_median = df.groupby('BS')['load'].median().to_dict()
    df['totbo51'] = df['BS'].map(BS_mean)
    df['totbo52'] = df['BS'].map(BS_std)
    df['totbo53'] = df['BS'].map(BS_median)
    df['previous'] = df.groupby('BS')[['Energy']].shift(1)
    df['previousload'] = df.groupby('BS')[['load']].shift(1)
    df['previous'] = df.groupby('BS')['previous'].fillna(method='ffill')
    df['previous'] = df.groupby('BS')['previous'].fillna(method='bfill')
    df['previousl'] = df.groupby('BS')['previousl'].fillna(method='bfill')
Feature_Engineering(df)

# Preprocessing

In [ ]:
train1= df[~df['Energy'].isna()]
test = df[df['Energy'].isna()]
train1=train1[train1['BS']!=838]
train1=train1[train1['BS']!=859]
train1=train1[train1['BS']!=835]
train1=train1[train1['BS']!=854]
train1 = train1.sort_values(by=['BS', 'Time'], ascending=[True, True]).reset_index(drop=True)
test = test.sort_values(by=['BS', 'Time'], ascending=[True, True]).reset_index(drop=True)
x=train1.drop(columns=['Energy',"ID","Time",'ESMode4','ESMode5','Frequency','CellName_Cell0','CellName_Cell3'])
y=train1['Energy']
test1=test.drop(columns=['Energy','Time',"ID",'ESMode4','ESMode5','Frequency','CellName_Cell0','CellName_Cell3'])

/tmp/ipykernel_115/3590102955.py:19: RuntimeWarning: invalid value encountered in double_scalars
  step = (next_value - current_value) / (end_idx - start_idx + 1)


In [ ]:
data1 = pd.DataFrame(train1['BS'].value_counts().reset_index())
datatest = pd.DataFrame(test['BS'].value_counts().reset_index().reset_index(drop=True))
l = list(data1['BS'].values)
mask = datatest["BS"].isin(l)
filtered_datatest = datatest[~mask]
filtered_datatest1 = datatest[mask]
l1=list(filtered_datatest['BS'])
l2=list(filtered_datatest1['BS'])
test_indexNBS=[]
for v in l2:
      test_indexNBS.append(test[test['BS']==v].index)
test_indexNBS = [item for sublist in test_indexNBS for item in sublist]
testNBS=test1.iloc[test_indexNBS]

In [ ]:
def Post_processing(datatest,data):
    lmin = pd.DataFrame(data.groupby('BS')[['Energy']].min()).reset_index()
    lmax = pd.DataFrame(data.groupby('BS')[['Energy']].max()).reset_index()
    l=np.unique(data['BS'])
    for i in range(23188):
        bs = datatest['BS'][i]
        if bs in l:
            if datatest['Energy'][i] > lmax['Energy'].loc[lmax['BS'] == bs].values[0]:
                datatest['Energy'][i] = lmax['Energy'].loc[lmax['BS'] == bs].values[0]
            elif datatest['Energy'][i] < lmin['Energy'].loc[lmin['BS'] == bs].values[0]:
                datatest['Energy'][i] = lmin['Energy'].loc[lmin['BS'] == bs].values[0]
    return datatest

# Model

In [ ]:
preds1CatBoost = []
def creat_model_catboost():
    params = {
        'loss_function': 'MAE',
        'iterations': 100000,
        'early_stopping_rounds': 500,
        'verbose': 1000,
        'l2_leaf_reg': 5,
        'max_depth': 8,"random_state":42,'task_type': 'GPU' }
    num_folds = 10
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    for fold_num, (train_index, test_index) in enumerate(kf.split(x)):
        X_train, X_val = x.iloc[train_index], x.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]
        dtrain = Pool(X_train, label=y_train)
        dval = Pool(X_val, label=y_val)
        dtest = Pool(testNBS)
        rf_regressor1 = CatBoostRegressor(**params)
        rf_regressor1.fit(dtrain, eval_set=dval)
        best_iteration = rf_regressor1.get_best_iteration()
        y_pred = rf_regressor1.predict(dval)
        feature_importance = rf_regressor1.get_feature_importance()
        feature_names = rf_regressor1.feature_names_
        sorted_idx = feature_importance.argsort()[::-1]
        top_n = 25
        plt.figure(figsize=(10, 6))
        plt.bar(range(top_n), feature_importance[sorted_idx][:top_n], align='center')
        plt.xticks(range(top_n), [feature_names[i] for i in sorted_idx][:top_n], rotation=90)
        plt.xlabel('Feature')
        plt.ylabel('Feature Importance')
        plt.title('Top 20 Feature Importances')
        plt.show()
        mae = mean_absolute_error(y_val, y_pred)
        print(f"Fold {fold_num + 1} MAE: {mae:.2f}")
        y_pred_test = rf_regressor1.predict(dtest)
        preds1CatBoost.append(y_pred_test)
    return preds1CatBoost,rf_regressor1

In [ ]:
def creat_model_xgboost():
    preds1xgb=[]
    scoore=[]
    num_folds = 10
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    for fold_num, (train_index, test_index) in enumerate(kf.split(x)):
        X_train, X_val = x.iloc[train_index], x.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]
        dtrain = DMatrix(X_train, label=y_train)
        dval = DMatrix(X_val, label=y_val)
        dtest=DMatrix(testNBS)
        params = {
            'objective': 'reg:squarederror',
        'eval_metric': 'mae','lambda': 5,
          'max_depth': 5,'random_state':42
        }
        bst = train(
        params,
        dtrain,num_boost_round=20000,
        early_stopping_rounds=200,
        verbose_eval=500,
        evals=[(dtrain, 'train'), (dval, 'val')]
        )
        plt.figure(figsize=(30, 30))
        xgb.plot_importance(bst,max_num_features=25)
        plt.show()
        best_iteration = bst.best_iteration
        scoore.append(bst.best_score)
        y_predtest = bst.predict(dtest,iteration_range=(0, best_iteration))
        preds1xgb.append(y_predtest)
    print(np.mean(scoore, axis=0))
    return preds1xgb,bst

In [ ]:
preds1CatBoost,rf_regressor1=creat_model()
tesCatBoosttNSS1=test.iloc[test_indexNBS]
y_predCatBoostNBS = np.mean(preds1CatBoost, axis=0)
tesCatBoosttNSS1['Energy']=y_predCatBoostNBS
tesCatBoosttNSS1=tesCatBoosttNSS1.reset_index(drop=True)
tesCatBoosttNSS1=Post_processing(tesCatBoosttNSS1,train1)
mergedCatBoost = pd.concat([train1, tesCatBoosttNSS1], axis= 0).sort_values(by=['BS', 'Time'], ascending=[True, True]).reset_index(drop=True)

/tmp/ipykernel_33/1391404788.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tesCatBoosttNSS1['Energy']=y_predCatBoostNBS


In [ ]:
preds1xgb,bst=creat_model_xgboost()
testxgbNSS1=test.iloc[test_indexNBS]
y_predxgbNBS = np.mean(preds1xgb, axis=0)
testxgbNSS1['Energy']=y_predxgbNBS
testxgbNSS1=testxgbNSS1.reset_index(drop=True)
testxgbNSS1=Post_processing(testxgbNSS1,train1)
mergedxgb = pd.concat([train1, testxgbNSS1], axis= 0).sort_values(by=['BS', 'Time'], ascending=[True, True]).reset_index(drop=True)

In [ ]:
def unknown_BS(df,dftest):
    train1= df[~df['Energy'].isna()]
    x1CatBoost=train1.drop(columns=['totbo51','totbo52','totbo53','previous','rolling_mean','Rolling_Std','Energy',"ID","Time",'BS','ESMode4','ESMode5','Frequency','CellName_Cell0','CellName_Cell3'])
    yCatBoost=train1['Energy']
    test2=dftest.drop(columns=['totbo51','totbo52','totbo53','previous','rolling_mean','Rolling_Std','Energy','Time',"ID",'BS','ESMode4','ESMode5','Frequency','CellName_Cell0','CellName_Cell3'])
    test_indexBS=[]
    for v in l1:
          test_indexBS.append(dftest[dftest['BS']==v].index)
    test_indexBS = [item for sublist in test_indexBS for item in sublist]
    testBS=test2.iloc[test_indexBS]
    testBSS=test.iloc[test_indexBS]
    return testBS,testBSS
testBSCatBoost,testBSSCatBoost=unknown_BS(mergedCatBoost,test)
testBSxgb,testBSSxgb=unknown_BS(mergedxgb,test)

In [ ]:
def greedy_grouping(values, ind,target_sum):
    groups = [[]]
    index = [[]]
    for i in range(len(values)):
        added = False
        for j in range(len(groups)):
            if sum(groups[j]) + values[i] <= target_sum:
                groups[j].append(values[i])
                index[j].append(ind[i])
                added = True
                break
        if not added:
            groups.append([values[i]])
            index.append([ind[i]])
    return index,groups
random.seed(42)
data=pd.DataFrame(train1['BS'].value_counts().reset_index())
data = data.sample(frac=1, random_state=42)
values = list(data['count'])
ind=list(data['BS'])
target_sum = 12000
index,result = greedy_grouping(values,ind ,target_sum)

In [ ]:
def creat_model_catboost_unknown_BS()
    preds2catboost = []
    params = {
        'loss_function': 'MAE',
        'iterations': 100000,
        'early_stopping_rounds': 500,
        'verbose': 1000,
        'l2_leaf_reg': 5,
        'max_depth': 5#jarib 5

    }

    num_folds = 10
    for fold_num in range(10):
        test_index=[]
        train_index=[]
        for v in index[fold_num]:
          test_index.append(train1[train1['BS']==v].index)
        test_index = [item for sublist in test_index for item in sublist]
        index1=index.copy()
        l=index1.pop(fold_num)
        index1 = [item for sublist in index1 for item in sublist]
        for v in index1:
          train_index.append(train1[train1['BS']==v].index)
        train_index = [item for sublist in train_index for item in sublist]
        print(len(train_index),len(test_index))
        X_train, X_val = x1CatBoost.iloc[train_index], x1CatBoost.iloc[test_index]
        y_train, y_val = yCatBoost.iloc[train_index], yCatBoost.iloc[test_index]
        dtrain = Pool(X_train, label=y_train)
        dval = Pool(X_val, label=y_val)
        dtest = Pool(testBSCatBoost)
        rf_regressor1 = CatBoostRegressor(**params)
        rf_regressor1.fit(dtrain, eval_set=dval)
        best_iteration = rf_regressor1.get_best_iteration()
        y_pred = rf_regressor1.predict(dval)
        feature_importance = rf_regressor1.get_feature_importance()
        feature_names = rf_regressor1.feature_names_
        sorted_idx = feature_importance.argsort()[::-1]
        top_n = 25
        plt.figure(figsize=(10, 6))
        plt.bar(range(top_n), feature_importance[sorted_idx][:top_n], align='center')
        plt.xticks(range(top_n), [feature_names[i] for i in sorted_idx][:top_n], rotation=90)
        plt.xlabel('Feature')
        plt.ylabel('Feature Importance')
        plt.title('Top 25 Feature Importances')
        plt.show()
        mae = mean_absolute_error(y_val, y_pred)
        print(f"Fold {fold_num + 1} MAE: {mae:.2f}")
        y_pred_test = rf_regressor1.predict(dtest)
        preds2catboost.append(y_pred_test)
        return preds2catboost,rf_regressor1
preds2catboost,rf_regressor1=creat_model_catboost_unknown_BS()

In [ ]:
def creat_model_xgboost_unknown_BS()
    preds2xgb=[]
    num_folds = 10
    for fold_num in range(10):
        test_index=[]
        train_index=[]
        for v in index[fold_num]:
          test_index.append(train1[train1['BS']==v].index)
        test_index = [item for sublist in test_index for item in sublist]
        index1=index.copy()
        l=index1.pop(fold_num)
        index1 = [item for sublist in index1 for item in sublist]
        for v in index1:
          train_index.append(train1[train1['BS']==v].index)
        train_index = [item for sublist in train_index for item in sublist]
        print(len(train_index),len(test_index))
        X_train, X_val = x1xgb.iloc[train_index], x1xgb.iloc[test_index]
        y_train, y_val = yxgb.iloc[train_index], yxgb.iloc[test_index]
        dtrain = DMatrix(X_train, label=y_train)
        dval = DMatrix(X_val, label=y_val)
        dtest=DMatrix(testBSxgb)
        params = {
            'objective': 'reg:squarederror',
        'eval_metric': 'mae','lambda': 5,
          'max_depth': 5
        }

        bst = train(
        params,
        dtrain,num_boost_round=10000,
        early_stopping_rounds=500,
        verbose_eval=500,
        evals=[(dtrain, 'train'), (dval, 'val')]
        )
        plt.figure(figsize=(30, 30))
        xgb.plot_importance(bst,max_num_features=25)
        plt.show()
        best_iteration = bst.best_iteration
        scoore.append(bst.best_score)
        y_predtest = bst.predict(dtest,iteration_range=(0, best_iteration))
        preds2xgb.append(y_predtest)
    print(np.mean(scoore, axis=0))
    return preds2xgb,bst
preds2xgb,bst=creat_model_xgboost_unknown_BS()

In [ ]:
y_predCatBoostBS = np.mean(preds2catboost, axis=0)
y_predxgbBS = np.mean(preds2xgb, axis=0)
testBSSCatBoost['Energy']=y_predCatBoostBS
testBSSxgb['Energy']=y_predxgbBS
predcatboost = pd.concat([tesCatBoosttNSS1, testBSSCatBoost], axis= 0).sort_values(by=['BS', 'Time'], ascending=[True, True]).reset_index(drop=True)
predxgb = pd.concat([testxgbNSS1, testBSSxgb], axis= 0).sort_values(by=['BS', 'Time'], ascending=[True, True]).reset_index(drop=True)
pred=predcatboost['Energy']*0.65+predxgb['Energy']*0.35

# Submission

In [ ]:
test['Energy']=predcatboost['Energy']
ss_df = ss_df[['ID']]
ss_df = ss_df.merge(test[['ID', 'Energy']], on='ID', how='left')
ss_df.to_csv("/kaggle/working/SampleSubmission (24).csv",index=False)
ss_df

,ID,Energy
0,2023-01-01 06:00:00_B_0,60.784380
1,2023-01-01 11:00:00_B_0,73.263328
2,2023-01-01 12:00:00_B_0,70.988538
3,2023-01-01 13:00:00_B_0,73.492854
4,2023-01-01 23:00:00_B_0,80.346904
...,...,...
26134,2023-01-02 19:00:00_B_1019,20.079494
26135,2023-01-02 20:00:00_B_1019,20.006799
26136,2023-01-02 21:00:00_B_1019,20.041855
26137,2023-01-02 22:00:00_B_1019,20.044033
